In [1]:
import pandas as pd

df = pd.read_csv('pacific-heights.csv')
df.head()

,address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,usecode,yearbuilt,zipcode
0,2243 Franklin St,2.0,2,1463,2/5/2016,1950000,37.795139,-122.425309,Pacific Heights,7,Condominium,1900,94109
1,2002 Pacific Ave APT 4,3.5,3,3291,1/22/2016,4200000,37.794429,-122.428513,Pacific Heights,7,Condominium,1961,94109
2,1945 Washington St APT 411,1.0,1,653,12/16/2015,665000,37.792472,-122.425281,Pacific Heights,3,Condominium,1987,94109
3,1896 Pacific Ave APT 802,2.5,2,2272,12/17/2014,2735000,37.794706,-122.426347,Pacific Heights,6,Condominium,1924,94109
4,1840 Washington St APT 603,1.0,1,837,12/2/2015,1050000,37.793212,-122.423744,Pacific Heights,3,Condominium,2012,94109


In [2]:
df.shape

(439, 13)

In [3]:
corr_matrix = df.corr()
corr_matrix["lastsoldprice"].sort_values(ascending=False)

lastsoldprice    1.000000
finishedsqft     0.840864
bathrooms        0.768179
totalrooms       0.670726
bedrooms         0.606579
zipcode          0.296032
latitude         0.126480
yearbuilt       -0.214365
longitude       -0.309462
Name: lastsoldprice, dtype: float64

In [4]:
df = pd.get_dummies(df, columns=['usecode'])
df.head()

,address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,yearbuilt,zipcode,usecode_Condominium,usecode_Cooperative,usecode_Duplex,usecode_MultiFamily2To4,usecode_SingleFamily,usecode_Townhouse
0,2243 Franklin St,2.0,2,1463,2/5/2016,1950000,37.795139,-122.425309,Pacific Heights,7,1900,94109,1,0,0,0,0,0
1,2002 Pacific Ave APT 4,3.5,3,3291,1/22/2016,4200000,37.794429,-122.428513,Pacific Heights,7,1961,94109,1,0,0,0,0,0
2,1945 Washington St APT 411,1.0,1,653,12/16/2015,665000,37.792472,-122.425281,Pacific Heights,3,1987,94109,1,0,0,0,0,0
3,1896 Pacific Ave APT 802,2.5,2,2272,12/17/2014,2735000,37.794706,-122.426347,Pacific Heights,6,1924,94109,1,0,0,0,0,0
4,1840 Washington St APT 603,1.0,1,837,12/2/2015,1050000,37.793212,-122.423744,Pacific Heights,3,2012,94109,1,0,0,0,0,0


In [5]:
corr_matrix = df.corr()
corr_matrix["lastsoldprice"].sort_values(ascending=False)

lastsoldprice              1.000000
finishedsqft               0.840864
bathrooms                  0.768179
totalrooms                 0.670726
usecode_SingleFamily       0.638482
bedrooms                   0.606579
zipcode                    0.296032
latitude                   0.126480
usecode_Duplex             0.006292
usecode_MultiFamily2To4   -0.019009
usecode_Townhouse         -0.025467
usecode_Cooperative       -0.025556
yearbuilt                 -0.214365
longitude                 -0.309462
usecode_Condominium       -0.598575
Name: lastsoldprice, dtype: float64

In [6]:
# Filter the dataset down to the most significant columns
df = df[['bathrooms', 'bedrooms', 'finishedsqft', 'totalrooms', 'usecode_Condominium', 'lastsoldprice']]
df.head()

,bathrooms,bedrooms,finishedsqft,totalrooms,usecode_Condominium,lastsoldprice
0,2.0,2,1463,7,1,1950000
1,3.5,3,3291,7,1,4200000
2,1.0,1,653,3,1,665000
3,2.5,2,2272,6,1,2735000
4,1.0,1,837,3,1,1050000


In [7]:
df = df.rename(index=str, columns={'usecode_Condominium': 'iscondo'})

In [8]:
corr_matrix = df.corr()
corr_matrix["lastsoldprice"].sort_values(ascending=False)

lastsoldprice    1.000000
finishedsqft     0.840864
bathrooms        0.768179
totalrooms       0.670726
bedrooms         0.606579
iscondo         -0.598575
Name: lastsoldprice, dtype: float64

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split

x = df.drop('lastsoldprice', axis=1)
y = df['lastsoldprice']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
model = RandomForestRegressor(random_state = 42)
model.fit(x_train, y_train)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [10]:
# Compute the model's R2 score
model.score(x_test, y_test)

0.7362903615843099

In [11]:
# Compute the mean R2 score across five folds
from sklearn.model_selection import cross_val_score

cross_val_score(model, x, y, cv=5).mean()

0.6308074678544755

In [12]:
# Use GridSearchCV to tune hyperparameters
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 50, 100],
    'max_features': [2, 3, 4],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [5, 10, 15]
}

grid_search = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5, verbose = 2)
grid_search.fit(x, y)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=10, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=3, min_samples_split

[CV]  bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=4, min_samples_split

[CV]  bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15 
[CV]  bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=3, min_samples_split=15, total=   0.0s
[CV] bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=4, min_samples_split

...

[CV]  bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=2, min_samples_leaf=3, min_sa

[CV]  bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=5 
[CV]  bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=10 
[CV]  bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_samples_split=10, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=3, min_samples_leaf=3, min_

[CV]  bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=5, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=10 
[CV]  bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=10, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=10 
[CV]  bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=10, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=10 
[CV]  bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=10, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=10 
[CV]  bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3, min_samples_split=10, total=   0.0s
[CV] bootstrap=False, max_depth=100, max_features=4, min_samples_leaf=3

[Parallel(n_jobs=1)]: Done 810 out of 810 | elapsed:   19.0s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'bootstrap': [True, False], 'max_depth': [10, 50, 100], 'max_features': [2, 3, 4], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [13]:
# Compute the mean R2 score across five folds using the best model
best_model = grid_search.best_estimator_

cross_val_score(best_model, x, y, cv=5).mean()

0.6945248837185962